In [2]:
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Read initial data
file_path='rtfDataSet.csv'
df = pd.read_csv(file_path)
print(df.head())

       0      1      2      3      4      5      6      7      8      9  ...  \
0 -0.536  0.292 -0.684  0.123 -0.118  0.346 -0.308 -0.113  0.401 -0.399  ...   
1 -0.496 -0.298 -0.529  0.073 -0.072  0.107 -0.316 -0.066 -0.168 -0.099  ...   
2  0.008 -0.031 -0.240  0.178 -0.120  0.317 -0.481  0.031 -0.077  0.063  ...   
3 -0.188 -0.180 -0.062 -0.104 -0.136 -0.061 -0.216 -0.143  0.068 -0.189  ...   
4  0.038  0.155 -0.203 -0.088 -0.084 -0.164 -0.145 -0.168  0.008  0.143  ...   

      61     62     63     64     65     66     67     68     69  label  
0 -0.135 -0.063 -0.410 -0.223 -0.599 -0.136 -0.329 -0.132 -0.266      0  
1  0.189  0.007 -0.362 -0.151 -0.338 -0.031 -0.159 -0.097 -0.131      0  
2 -0.035  0.101 -0.098 -0.172 -0.290  0.033 -0.342 -0.321 -0.238      0  
3  0.146 -0.145 -0.527 -0.292 -0.663 -0.078 -0.194 -0.151 -0.268      0  
4  0.019 -0.084  0.164 -0.165 -0.126 -0.112  0.029 -0.138  0.148      1  

[5 rows x 71 columns]


In [4]:
#Reduce to sets and normalized
set_2=[2, 5, 9, 10, 16, 21, 28, 34, 38, 48, 49, 55, 61, 62, 64]
set_3=[1, 2, 7, 8, 9, 10, 19, 23, 28, 34, 35, 36, 42, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 63, 64, 65, 67]

reduced_df_2=df.iloc[:,set_2]
reduced_df_3=df.iloc[:,set_3]

y = df.iloc[:, -1]

scaler = MinMaxScaler()

reduced_df_2 = scaler.fit_transform(reduced_df_2)
reduced_df_3 = scaler.fit_transform(reduced_df_3)

#Array of sets
reduced_array=[reduced_df_2,reduced_df_3]

In [7]:
#Creacion dinamica de folds y clasificacion y metricas
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter=1
epochs = [1,50,100,150,200,250,300,350,400,450,500]

#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(6, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))


Set:  1
12/12 [==============================] - 1s 2ms/step - loss: 0.7003 - accuracy: 0.5112
Epoch 1/50
12/12 [==============================] - 1s 2ms/step - loss: 0.6996 - accuracy: 0.5112
Epoch 2/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6946 - accuracy: 0.5112
Epoch 3/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6978 - accuracy: 0.5112
Epoch 4/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6939 - accuracy: 0.5112
Epoch 5/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6952 - accuracy: 0.5112
Epoch 6/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6935 - accuracy: 0.5112
Epoch 7/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6952 - accuracy: 0.5112
Epoch 8/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6936 - accuracy: 0.5112
Epoch 9/50
12/12 [==============================] - 0s 2ms/step - loss: 0.6966 - accuracy: 0.5112
Epoch 10/50
12/12 [====

KeyboardInterrupt: 